In [11]:
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union

import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.utils.tensorboard
from torch import Tensor
from tqdm import tqdm

import delu

import sys
import os
os.environ['PROJECT_DIR'] = '/Users/timgritsaev/Desktop/tabular/tabular-hpo'
sys.path.append('/Users/timgritsaev/Desktop/tabular/tabular-hpo')
import lib
from lib import KWArgs

import json

from loss_landscapes.metrics.metric import Metric
from loss_landscapes.model_interface.model_wrapper import ModelWrapper


import loss_landscapes
import loss_landscapes.metrics
import copy
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
plt.rcParams['savefig.format'] = 'pdf'


@dataclass(frozen=True)
class Config:
    seed: int
    data: Union[lib.Dataset[np.ndarray], KWArgs]  # lib.data.build_dataset
    model: Union[nn.Module, KWArgs]  # Model
    optimizer: Union[torch.optim.Optimizer, KWArgs]  # lib.deep.make_optimizer
    batch_size: int
    patience: Optional[int]
    n_epochs: Union[int, float]


def _patch_config(c: lib.JSONDict):
    # update config format
    if isinstance(c['model'], dict) and 'backbone' not in c['model']:
        assert 'num_embeddings' in c
        c['model'] = {
            'num_embeddings': c.pop('num_embeddings'),
            'backbone': c.pop('model'),
        }
        

def create_all(path_to_config, path_to_model, steps=10, distance=0.001):

    class Model(nn.Module):
        def __init__(
            self,
            *,
            n_num_features: int,
            n_bin_features: int,
            cat_cardinalities: list[int],
            n_classes: Optional[int],
            num_embeddings: Optional[dict],  # lib.deep.ModuleSpec
            backbone: dict,  # lib.deep.ModuleSpec
        ) -> None:
            assert n_num_features or n_bin_features or cat_cardinalities
            if num_embeddings is not None:
                assert n_num_features
            assert backbone['type'] in ['MLP', 'ResNet']
            super().__init__()

            if num_embeddings is None:
                self.m_num = nn.Identity() if n_num_features else None
                d_num = n_num_features
            else:
                self.m_num = lib.make_module(num_embeddings, n_features=n_num_features)
                d_num = n_num_features * num_embeddings['d_embedding']
            self.m_bin = nn.Identity() if n_bin_features else None
            self.m_cat = lib.OneHotEncoder(cat_cardinalities) if cat_cardinalities else None
            self.backbone = lib.make_module(
                backbone,
                d_in=d_num + n_bin_features + sum(cat_cardinalities),
                d_out=lib.get_d_out(n_classes),
            )
            self.flat = True

        def forward(
            self,
            *,
            x_num: Optional[Tensor],
            x_bin: Optional[Tensor],
            x_cat: Optional[Tensor],
        ) -> Tensor:
            x = []
            for module, x_ in [
                (self.m_num, x_num),
                (self.m_bin, x_bin),
                (self.m_cat, x_cat),
            ]:
                if x_ is None:
                    assert module is None
                else:
                    assert module is not None
                    x.append(module(x_))
            del x_  # type: ignore[code]
            if self.flat:
                x = torch.cat([x_.flatten(1, -1) for x_ in x], dim=1)
            else:
                # for Transformer-like backbones (currently unsupported)
                assert all(x_.ndim == 3 for x_ in x)
                x = torch.cat(x, dim=1)

            x = self.backbone(x)
            return x
        

    class Model__(nn.Module):
        def __init__(
            self,
            *,
            n_num_features: int,
            n_bin_features: int,
            cat_cardinalities: list[int],
            n_classes: Optional[int],
            num_embeddings: Optional[dict],  # lib.deep.ModuleSpec
            backbone: dict,  # lib.deep.ModuleSpec
        ) -> None:
            assert n_num_features or n_bin_features or cat_cardinalities
            if num_embeddings is not None:
                assert n_num_features
            assert backbone['type'] in ['MLP', 'ResNet']
            super().__init__()

            if num_embeddings is None:
                self.m_num = nn.Identity() if n_num_features else None
                d_num = n_num_features
            else:
                self.m_num = lib.make_module(num_embeddings, n_features=n_num_features)
                d_num = n_num_features * num_embeddings['d_embedding']
            self.m_bin = nn.Identity() if n_bin_features else None
            self.m_cat = lib.OneHotEncoder(cat_cardinalities) if cat_cardinalities else None
            self.backbone = lib.make_module(
                backbone,
                d_in=d_num + n_bin_features + sum(cat_cardinalities),
                d_out=lib.get_d_out(n_classes),
            )
            self.flat = True
            self.current_dataset = 'train'

        def forward(
            self,
            X
        ) -> Tensor:
            x = []
            x_num = None if dataset.X_num is None else dataset.X_num[self.current_dataset][X]
            x_bin = None if dataset.X_bin is None else dataset.X_bin[self.current_dataset][X]
            x_cat = None if dataset.X_cat is None else dataset.X_cat[self.current_dataset][X]
            for module, x_ in [
                (self.m_num, x_num),
                (self.m_bin, x_bin),
                (self.m_cat, x_cat),
            ]:
                if x_ is None:
                    assert module is None
                else:
                    assert module is not None
                    x.append(module(x_))
            del x_  # type: ignore[code]
            if self.flat:
                x = torch.cat([x_.flatten(1, -1) for x_ in x], dim=1)
            else:
                # for Transformer-like backbones (currently unsupported)
                assert all(x_.ndim == 3 for x_ in x)
                x = torch.cat(x, dim=1)

            x = self.backbone(x)
            return x
        
        
    def apply_model(part, idx):
        return model(
            x_num=None if dataset.X_num is None else dataset.X_num[part][idx],
            x_bin=None if dataset.X_bin is None else dataset.X_bin[part][idx],
            x_cat=None if dataset.X_cat is None else dataset.X_cat[part][idx],
        ).squeeze(-1)
        
        
    def apply_model__(part, idx):
        model.current_dataset = part
        return model(idx).squeeze(-1)


    def are_valid_predictions(predictions: dict[str, np.ndarray]) -> bool:
        return all(np.isfinite(x).all() for x in predictions.values())
        
        
    @torch.inference_mode()
    def evaluate(parts: list[str], eval_batch_size: int):
        model.eval()
        predictions = {}
        for part in parts:
            while eval_batch_size:
                try:
                    predictions[part] = (
                        torch.cat(
                            [
                                apply_model__(part, idx)
                                for idx in torch.arange(
                                    dataset.size(part), device=device
                                ).split(eval_batch_size)
                            ]
                        )
                        .cpu()
                        .numpy()
                    )
                except RuntimeError as err:
                    if not lib.is_oom_exception(err):
                        raise
                    eval_batch_size //= 2
                    # logger.warning(f'eval_batch_size = {eval_batch_size}')
                else:
                    break
            if not eval_batch_size:
                RuntimeError('Not enough memory even for eval_batch_size=1')
        metrics = (
            dataset.calculate_metrics(predictions, report['prediction_type'])
            if are_valid_predictions(predictions)
            else {x: {'score': -999999.0} for x in predictions}
        )
        return metrics, predictions, eval_batch_size


    class TrainLoss(Metric):
        """ Computes a specified loss function over specified input-output pairs. """
        def __init__(self, loss_fn, ids: torch.Tensor):
            super().__init__()
            self.loss_fn = loss_fn
            self.ids = ids

        def __call__(self, model_wrapper: ModelWrapper) -> float:
            # model_wrapper.current_dataset = 'val'
            # loss = self.loss_fn(model_wrapper.forward(torch.arange(dataset.size('val'))).squeeze(-1), val_y).item()
            model_wrapper.current_dataset = 'train'
            loss = self.loss_fn(model_wrapper.forward(torch.arange(dataset.size('train'))).squeeze(-1), train_y).item()
            # model_wrapper.current_dataset = 'train'
            return loss
        
        
    class ValLoss(Metric):
        """ Computes a specified loss function over specified input-output pairs. """
        def __init__(self, loss_fn, ids: torch.Tensor):
            super().__init__()
            self.loss_fn = loss_fn
            self.ids = ids

        def __call__(self, model_wrapper: ModelWrapper) -> float:
            model_wrapper.current_dataset = 'val'
            loss = self.loss_fn(model_wrapper.forward(torch.arange(dataset.size('val'))).squeeze(-1), val_y).item()
            model_wrapper.current_dataset = 'train'
            return loss
        
        
    class TestLoss(Metric):
        """ Computes a specified loss function over specified input-output pairs. """
        def __init__(self, loss_fn, ids: torch.Tensor):
            super().__init__()
            self.loss_fn = loss_fn
            self.ids = ids

        def __call__(self, model_wrapper: ModelWrapper) -> float:
            model_wrapper.current_dataset = 'test'
            loss = self.loss_fn(model_wrapper.forward(torch.arange(dataset.size('test'))).squeeze(-1), test_y).item()
            model_wrapper.current_dataset = 'train'
            return loss
        
    if path_to_config[-2:] == "pt":
        config = torch.load(path_to_config)['report']['best']['config']
    else:
        file_in = open(path_to_config, "r")
        config = json.loads(file_in.read())
    # print(config)
    _patch_config(config)
    C = lib.make_config(Config, config)
    report = lib.create_report(config)

    delu.improve_reproducibility(C.seed)
    device = lib.get_device()

    dataset = (C.data if isinstance(C.data, lib.Dataset) else lib.build_dataset(**C.data)).to_torch(device)

    loss_fn = lib.get_loss_fn(dataset.task_type)

    model = Model__(
                n_num_features=dataset.n_num_features,
                n_bin_features=dataset.n_bin_features,
                cat_cardinalities=dataset.cat_cardinalities(),
                n_classes=dataset.n_classes(),
                **C.model,
            )
    model.load_state_dict(torch.load(path_to_model))
    model.eval()

    # os.system('clear')

    report['n_parameters'] = lib.get_n_parameters(model)
    report['prediction_type'] = None if dataset.is_regression else 'logits'
    
    train_model = copy.deepcopy(model)
    train_model.current_dataset = 'train'
    val_model = copy.deepcopy(model)
    val_model.current_dataset = 'val'
    test_model = copy.deepcopy(model)
    test_model.current_dataset = 'test'

    train_x = iter(delu.data.make_index_dataloader(dataset.size('train'), C.batch_size, shuffle=False)).__next__()
    val_x = iter(delu.data.make_index_dataloader(dataset.size('val'), C.batch_size, shuffle=False)).__next__()
    test_x = iter(delu.data.make_index_dataloader(dataset.size('test'), C.batch_size, shuffle=False)).__next__()

    train_y = dataset.Y['train'].to(torch.long if dataset.is_multiclass else torch.float)
    val_y = dataset.Y['val'].to(torch.long if dataset.is_multiclass else torch.float)
    test_y = dataset.Y['test'].to(torch.long if dataset.is_multiclass else torch.float)

    train_metric = TrainLoss(loss_fn, train_x)
    val_metric = ValLoss(loss_fn, val_x)
    test_metric = TestLoss(loss_fn, test_x)
    

    def plot_log1ploss_landscape(steps, distance):
        train_loss_data_fin = loss_landscapes.random_plane(train_model, train_metric, distance, steps, deepcopy_model=True)
        train_loss_data_fin = (train_loss_data_fin ** 0.5) if loss_fn is torch.nn.functional.mse_loss else train_loss_data_fin

        val_loss_data_fin = loss_landscapes.random_plane(val_model, val_metric, distance, steps, deepcopy_model=True)
        val_loss_data_fin = (val_loss_data_fin ** 0.5) if loss_fn is torch.nn.functional.mse_loss else train_loss_data_fin

        test_loss_data_fin = loss_landscapes.random_plane(test_model, test_metric, distance, steps, deepcopy_model=True)
        test_loss_data_fin = (test_loss_data_fin ** 0.5) if loss_fn is torch.nn.functional.mse_loss else train_loss_data_fin

        fig = go.Figure(data=[go.Surface(z=np.log1p(train_loss_data_fin)),
                              go.Surface(z=np.log1p(val_loss_data_fin)),
                              go.Surface(z=np.log1p(test_loss_data_fin))])
        fig.update_layout(autosize=False, width=500, height=500, margin=dict(l=65, r=50, b=65, t=90))
        fig.show()
        
    def plot_loss_landscapes(steps, distance):
        train_loss_data_fin = loss_landscapes.random_plane(train_model, train_metric, distance, steps, deepcopy_model=True)
        train_loss_data_fin = (train_loss_data_fin ** 0.5) if loss_fn is torch.nn.functional.mse_loss else train_loss_data_fin

        val_loss_data_fin = loss_landscapes.random_plane(val_model, val_metric, distance, steps, deepcopy_model=True)
        val_loss_data_fin = (val_loss_data_fin ** 0.5) if loss_fn is torch.nn.functional.mse_loss else train_loss_data_fin

        test_loss_data_fin = loss_landscapes.random_plane(test_model, test_metric, distance, steps, deepcopy_model=True)
        test_loss_data_fin = (test_loss_data_fin ** 0.5) if loss_fn is torch.nn.functional.mse_loss else train_loss_data_fin

        X = np.arange(-steps / 2, steps / 2)
        Y = np.arange(-steps / 2, steps / 2)
        fig = go.Figure(data=[go.Surface(z=train_loss_data_fin, x=X, y=Y)])
        fig.update_scenes(xaxis_title_text='dir_one', yaxis_title_text='dir_two')
        fig.update_layout(title='train', autosize=False, width=500, height=500, margin=dict(l=65, r=50, b=65, t=90))
        fig.show()
        fig.write_image(f'images/landscape_train_dist{distance}.pdf', format='pdf')

        fig = go.Figure(data=[go.Surface(z=val_loss_data_fin, x=X, y=Y)])
        fig.update_scenes(xaxis_title_text='dir_one', yaxis_title_text='dir_two', zaxis_title_text='RMSE')
        fig.update_layout(title='val', autosize=False, width=500, height=500, margin=dict(l=65, r=50, b=65, t=90))
        fig.write_image(f'images/landscape_val_dist{distance}.pdf', format='pdf')

        fig = go.Figure(data=[go.Surface(z=test_loss_data_fin, x=X, y=Y)])
        fig.update_scenes(xaxis_title_text='dir_one', yaxis_title_text='dir_two', zaxis_title_text='RMSE')
        fig.update_layout(title='test', autosize=False, width=500, height=500, margin=dict(l=65, r=50, b=65, t=90))
        fig.write_image(f'images/landscape_test_dist{distance}.pdf', format='pdf')

    
    plot_loss_landscapes(steps, distance)

In [12]:
DATASET = 'california'
path_to_config = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/exp/mlp/' + DATASET + '/0-tuning/checkpoint.pt'
path_to_model = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/saved_models/optuna_' + DATASET + '_0.pt'
create_all(path_to_config, path_to_model, steps=50, distance=0.1)

/Users/timgritsaev/Desktop/tabular/tabular-hpo/lib/util.py:237: UserWarning:

The key "function" will be missing in the report. Reason: '/private/var/folders/65/1vwp97n52f5bhqv1kd377bw40000gn/T/ipykernel_4274/81916396.py' is not in the subpath of '/Users/timgritsaev/Desktop/tabular/tabular-hpo' OR one path is relative and the other is absolute.

0:55:30 | Using cached features: build_dataset__california__quantile__None__standard__0__f7706b82c7264c3b33fedc53664ebacf.pickle


----------------------------------------------------------------------------------------------------
{'seed': 0,
 'data': {'seed': 0,
          'cache': True,
          'path': ':data/california',
          'num_policy': 'quantile',
          'cat_policy': None,
          'y_policy': 'standard'},
 'model': {'num_embeddings': None,
           'backbone': {'type': 'MLP',
                        'n_blocks': 2,
                        'd_layer': 238,
                        'dropout': 0.23395140372463127,
                        'activation': 'ReLU'}},
 'optimizer': {'type': 'AdamW', 'lr': 0.0009213130217887258, 'weight_decay': 3.911847192982179e-05},
 'batch_size': 256,
 'patience': 16,
 'n_epochs': inf}
----------------------------------------------------------------------------------------------------


In [8]:
DATASET = 'california'
path_to_config = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/exp/mlp/' + DATASET + '/0-tuning/checkpoint.pt'
path_to_model = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/saved_models/optuna_' + DATASET + '_0.pt'
create_all(path_to_config, path_to_model, steps=50, distance=0.01)

0:37:59 | Using cached features: build_dataset__california__quantile__None__standard__0__f7706b82c7264c3b33fedc53664ebacf.pickle


----------------------------------------------------------------------------------------------------
{'seed': 0,
 'data': {'seed': 0,
          'cache': True,
          'path': ':data/california',
          'num_policy': 'quantile',
          'cat_policy': None,
          'y_policy': 'standard'},
 'model': {'num_embeddings': None,
           'backbone': {'type': 'MLP',
                        'n_blocks': 2,
                        'd_layer': 238,
                        'dropout': 0.23395140372463127,
                        'activation': 'ReLU'}},
 'optimizer': {'type': 'AdamW', 'lr': 0.0009213130217887258, 'weight_decay': 3.911847192982179e-05},
 'batch_size': 256,
 'patience': 16,
 'n_epochs': inf}
----------------------------------------------------------------------------------------------------


In [22]:
DATASET = 'churn'
path_to_config = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/exp/mlp/' + DATASET + '/0-tuning/checkpoint.pt'
path_to_model = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/saved_models/optuna_' + DATASET + '_0.pt'
create_all(path_to_config, path_to_model)

/Users/timgritsaev/Desktop/tabular/tabular-hpo/lib/util.py:237: UserWarning:

The key "function" will be missing in the report. Reason: '/private/var/folders/65/1vwp97n52f5bhqv1kd377bw40000gn/T/ipykernel_33734/486199965.py' is not in the subpath of '/Users/timgritsaev/Desktop/tabular/tabular-hpo' OR one path is relative and the other is absolute.

0:15:08 | Using cached features: build_dataset__churn__quantile__ordinal__None__0__cfd36cf52f69e9043707914b6ce4682a.pickle


----------------------------------------------------------------------------------------------------
{'seed': 0,
 'data': {'seed': 0,
          'cache': True,
          'path': ':data/churn',
          'num_policy': 'quantile',
          'cat_policy': 'ordinal',
          'y_policy': None},
 'model': {'num_embeddings': None,
           'backbone': {'type': 'MLP',
                        'n_blocks': 3,
                        'd_layer': 544,
                        'dropout': 0.4323534887560793,
                        'activation': 'ReLU'}},
 'optimizer': {'type': 'AdamW', 'lr': 0.0004580779164922931, 'weight_decay': 3.617216368553555e-06},
 'batch_size': 256,
 'patience': 16,
 'n_epochs': inf}
----------------------------------------------------------------------------------------------------
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1.])
tensor([1., 0., 0., 1., 1., 1., 0., 0., 1., 0.])
dir_one 8
dir_one tensor([[0.4963, 0.7682, 0.088

In [5]:
DATASET = 'gesture'
path_to_config = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/exp/mlp/' + DATASET + '/0-tuning/checkpoint.pt'
path_to_model = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/saved_models/optuna_' + DATASET + '_0.pt'
create_all(path_to_config, path_to_model, distance=0.1)

/Users/timgritsaev/Desktop/tabular/tabular-hpo/lib/util.py:237: UserWarning:

The key "function" will be missing in the report. Reason: '/private/var/folders/65/1vwp97n52f5bhqv1kd377bw40000gn/T/ipykernel_2797/2891418144.py' is not in the subpath of '/Users/timgritsaev/Desktop/tabular/tabular-hpo' OR one path is relative and the other is absolute.

0:05:57 | Using cached features: build_dataset__gesture__quantile__None__None__0__a4b7fd283c9b282df4cc1225d2489303.pickle


----------------------------------------------------------------------------------------------------
{'seed': 0,
 'data': {'seed': 0,
          'cache': True,
          'path': ':data/gesture',
          'num_policy': 'quantile',
          'cat_policy': None,
          'y_policy': None},
 'model': {'num_embeddings': None,
           'backbone': {'type': 'MLP',
                        'n_blocks': 6,
                        'd_layer': 939,
                        'dropout': 0.0819818600292229,
                        'activation': 'ReLU'}},
 'optimizer': {'type': 'AdamW',
               'lr': 0.00032048018345648356,
               'weight_decay': 2.178217166538039e-05},
 'batch_size': 128,
 'patience': 16,
 'n_epochs': inf}
----------------------------------------------------------------------------------------------------
tensor([4, 2, 4, 1, 0, 2, 2, 0, 3, 2])
tensor([4, 2, 0, 4, 0, 4, 4, 2, 4, 2])
tensor([0, 0, 2, 0, 4, 0, 2, 2, 2, 4])
dir_one 14
dir_one tensor([[0.4963, 0.7682, 0.0885

In [3]:
DATASET = 'diamond'
path_to_config = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/exp/mlp/' + DATASET + '/0-tuning/checkpoint.pt'
path_to_model = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/saved_models/optuna_' + DATASET + '_0.pt'
create_all(path_to_config, path_to_model, distance=0.1)

----------------------------------------------------------------------------------------------------
{'seed': 0,
 'data': {'seed': 0,
          'cache': True,
          'path': ':data/diamond',
          'num_policy': 'quantile',
          'cat_policy': 'ordinal',
          'y_policy': 'standard'},
 'model': {'num_embeddings': None,
           'backbone': {'type': 'MLP',
                        'n_blocks': 3,
                        'd_layer': 435,
                        'dropout': 0.2785541541043303,
                        'activation': 'ReLU'}},
 'optimizer': {'type': 'AdamW', 'lr': 0.000689152147641599, 'weight_decay': 0.0},
 'batch_size': 512,
 'patience': 16,
 'n_epochs': inf}
----------------------------------------------------------------------------------------------------


/Users/timgritsaev/Desktop/tabular/tabular-hpo/lib/util.py:237: UserWarning: The key "function" will be missing in the report. Reason: '/private/var/folders/65/1vwp97n52f5bhqv1kd377bw40000gn/T/ipykernel_2797/2891418144.py' is not in the subpath of '/Users/timgritsaev/Desktop/tabular/tabular-hpo' OR one path is relative and the other is absolute.
  warnings.warn(
0:00:00 | Using cached features: build_dataset__diamond__quantile__ordinal__standard__0__73eb7ffa61642f295a3ae6df6c4a08ac.pickle


tensor([-0.5621, -0.8780, -0.7785,  1.3097, -0.0312, -0.5247,  2.2776,  0.9024,
        -0.8872,  1.7469])
tensor([-0.3702, -0.1541, -0.5501, -0.3787,  0.1105, -0.5659, -0.8295,  1.8532,
        -0.2184, -0.0114])
tensor([ 0.2027,  0.6276,  0.3979,  1.2170,  0.1424, -0.7558, -0.7883, -0.2772,
         3.7132, -0.8591])
dir_one 8
dir_one tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074],
        [0.9527, 0.0362, 0.1852, 0.3734, 0.3051],
        [0.6984, 0.5675, 0.8352, 0.2056, 0.5932],
        [0.5460, 0.1872, 0.0340, 0.9442, 0.8802],
        [0.0237, 0.4910, 0.1235, 0.1143, 0.4725]])
start_point tensor([[ 0.0456,  0.0140,  0.0463, -0.0101,  0.1177],
        [-0.2230, -0.0137, -0.0114, -0.2229, -0.3695],
        [ 0.1029, -0.0199,  0.0087, -0.0585, -0.0711],
        [-0.0694, -0.0886,  0.1605, -0.0546, -0.2407],
        [ 0.1032,  0.0236,  0.1340,  0.1433, -0.1868]])
dir_one 8
dir_one tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074],
        [0.9527, 0.0362, 0.1852, 0.3734, 0.3051],
    

In [4]:
DATASET = 'diamond'
path_to_config = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/exp/mlp/' + DATASET + '/0-tuning/checkpoint.pt'
path_to_model = '/Users/timgritsaev/Desktop/tabular/tabular-hpo/saved_models/optuna_' + DATASET + '_0.pt'
create_all(path_to_config, path_to_model, distance=0.01)

/Users/timgritsaev/Desktop/tabular/tabular-hpo/lib/util.py:237: UserWarning:

The key "function" will be missing in the report. Reason: '/private/var/folders/65/1vwp97n52f5bhqv1kd377bw40000gn/T/ipykernel_2797/2891418144.py' is not in the subpath of '/Users/timgritsaev/Desktop/tabular/tabular-hpo' OR one path is relative and the other is absolute.

0:02:27 | Using cached features: build_dataset__diamond__quantile__ordinal__standard__0__73eb7ffa61642f295a3ae6df6c4a08ac.pickle


----------------------------------------------------------------------------------------------------
{'seed': 0,
 'data': {'seed': 0,
          'cache': True,
          'path': ':data/diamond',
          'num_policy': 'quantile',
          'cat_policy': 'ordinal',
          'y_policy': 'standard'},
 'model': {'num_embeddings': None,
           'backbone': {'type': 'MLP',
                        'n_blocks': 3,
                        'd_layer': 435,
                        'dropout': 0.2785541541043303,
                        'activation': 'ReLU'}},
 'optimizer': {'type': 'AdamW', 'lr': 0.000689152147641599, 'weight_decay': 0.0},
 'batch_size': 512,
 'patience': 16,
 'n_epochs': inf}
----------------------------------------------------------------------------------------------------
tensor([-0.5621, -0.8780, -0.7785,  1.3097, -0.0312, -0.5247,  2.2776,  0.9024,
        -0.8872,  1.7469])
tensor([-0.3702, -0.1541, -0.5501, -0.3787,  0.1105, -0.5659, -0.8295,  1.8532,
        -0.2184, -0